In [1]:
from postgres_connect import *

In [2]:
path = 'tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)

Tunnel Started
Connection Made


In [3]:
query_path = 'rider_pip_closure.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
cursor = conn.cursor()
cursor.execute(custom_query)
conn.commit()
# conn.close()

In [4]:
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)

In [5]:
df

,key,pip_date,rider_id,shipping_city,rider_name,hub,vendor_name,rider_age,need_improvement,starting_fasr,closing_fasr,pip_status,pip_result,update_time
0,20240304-K_Srikanth_4_1701665808,2024-03-04,K_Srikanth_4_1701665808,Hyderabad,K Srikanth,DS HYD TRNK,Blitz_Self,83,Morning Prepaid (64%),80.0,89.0,closed,passed,2024-03-12 13:42:12.686691
1,20240304-Lokesh_JPS_8368772866_023,2024-03-04,Lokesh_JPS_8368772866_023,Jaipur,Lokesh_JPS,DS JPR NPRG,JPS Group of Management,15,Morning Prepaid (76%),65.0,61.0,closed,failed,2024-03-12 13:42:12.686691
2,20240304-Ibrahim_8657617894_03,2024-03-04,Ibrahim_8657617894_03,Mumbai,Rehan Sakware,DS BOM KRL,Team HR,43,Morning COD(48%),73.0,71.0,closed,failed,2024-03-12 13:42:12.686691
3,20240304-Ajit_9930585435_03,2024-03-04,Ajit_9930585435_03,Mumbai,Ajit nikam,DS BOM KRL,Rapydex,10,Morning COD(51%),77.0,79.0,closed,passed,2024-03-12 13:42:12.686691
4,20240304-Munir_shaikh__3,2024-03-04,Munir_shaikh__3,Mumbai,Munir shaikh,DS BOM KRL,Team HR,111,Morning COD(53%),81.0,87.0,closed,passed,2024-03-12 13:42:12.686691
5,20240304-Nitesh_9769326235_03,2024-03-04,Nitesh_9769326235_03,Mumbai,Nitesh Sharma,DS BOM KRL,Team HR,60,Morning COD(55%),82.0,93.0,closed,passed,2024-03-12 13:42:12.686691
6,20240304-DS_BOM_KRL_28062022_003,2024-03-04,DS_BOM_KRL_28062022_003,Mumbai,Salauddin Shaikh,DS BOM KRL,Ambika Logistics,228,Morning Prepaid (83%),83.0,100.0,closed,passed,2024-03-12 13:42:12.686691
7,20240304-Anil_9987304419_03,2024-03-04,Anil_9987304419_03,Mumbai,Anil denish w,DS BOM SWRI,Unskill,72,Morning COD(40%),75.0,73.0,closed,failed,2024-03-12 13:42:12.686691
8,20240304-Irfan_8689929948_022,2024-03-04,Irfan_8689929948_022,Mumbai,Irfan Shaikh,DS BOM SWRI,Team HR,10,Morning Prepaid (82%),77.0,89.0,closed,passed,2024-03-12 13:42:12.686691
9,20240304-K_8884566500_07,2024-03-04,K_8884566500_07,Bangalore,K Goutham,DS BLR HBBL,Blitz_Self,38,Morning Prepaid (77%),71.0,80.0,closed,passed,2024-03-12 13:42:12.686691


In [6]:
query_path = 'rider_pip_closure_performance.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
df_pip = pd.read_sql(custom_query,conn)
df_pip.head()

,pip_date,shipping_city,rider_id,rider_name,hub,performance_type,threshold,monday,tuesday,wednesday,thursday,friday,saturday,sunday,avg_fasr,status
0,2024-03-04,Bangalore,K_8884566500_07,K Goutham,DS BLR HBBL,Morning-Prepaid,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail
1,2024-03-04,Bangalore,K_8884566500_07,K Goutham,DS BLR HBBL,Morning-COD,0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail
2,2024-03-04,Bangalore,K_8884566500_07,K Goutham,DS BLR HBBL,Evening-Prepaid,0.80,0.93,0.9,0.77,0.66,0.92,0.77,0.85,0.85,pass
3,2024-03-04,Bangalore,K_8884566500_07,K Goutham,DS BLR HBBL,Evening-COD,0.50,1.00,1.0,NaN,0.50,1.00,0.66,0.57,0.66,pass
4,2024-03-04,Delhi,Ashish_9711286588_05,Ashish KumarJPS1,DS DEL DWK,Morning-Prepaid,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail


In [9]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Filter failed riders
failed_riders = df[df['pip_result'] == 'failed']

# Group by city and hub
grouped_data = failed_riders.groupby(['shipping_city', 'hub']).size().reset_index(name='failed_count')

# Group by city to get total rider count
total_riders = df.groupby(['shipping_city', 'hub']).size().reset_index(name='total_count')

# Merge the two dataframes on city
result_df = pd.merge(total_riders, grouped_data, on=['shipping_city','hub'], how='left')

# Calculate the failed rate without decimals
result_df['failed_rate'] = (result_df['failed_count'] / result_df['total_count']) * 100
result_df['failed_rate'] = result_df['failed_rate'].fillna(0)
result_df['failed_rate'] = result_df['failed_rate'].astype(int)

# Email configuration
sender_email = "akshay@blitznow.in"
sender_password = "Outofblitz1!"

# Create the message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = "operations@blitznow.in"
message['Subject'] = f"Rider PIP Closure Report Week - {failed_riders['pip_date'].max()}"

# Create an Excel file with two sheets
excel_file_path = 'failed_riders_report.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Write the failed_riders DataFrame to the first sheet
    failed_riders.to_excel(writer, sheet_name='Failed_Riders_Data', index=False)

    # Write the df_pip DataFrame to the second sheet
    df_pip.to_excel(writer, sheet_name='Additional_Sheet', index=False)

# Attach the Excel file
with open(excel_file_path, 'rb') as excel_file:
    attachment = MIMEApplication(excel_file.read(), _subtype='xlsx')
    attachment.add_header('Content-Disposition', 'attachment', filename='pip_closure_report.xlsx')
    message.attach(attachment)

# Create an HTML table for the email body with formatted headers
html_table = result_df.to_html(index=False, border=1, classes='table', escape=False, justify='center')
html_table = html_table.replace('<th>', '<th style="font-weight:bold; background-color:lightblue; text-align:center;">').replace('_', ' ').upper()

# Add the table to the email body
body = f"""\
<html>
  <head></head>
  <body>
    <p>Dear recipients,</p>
    <p>Please find attached the report for RIDER PIP Result for the week - {failed_riders['pip_date'].max()}.</p>
    <p>Summary:</p>
    {html_table}
    <p>Best regards,<br>Your Name</p>
  </body>
</html>
"""

message.attach(MIMEText(body, 'html'))
recipients = "operations@blitznow.in"
# Connect to the SMTP server and send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipients, message.as_string())

print("Email sent successfully.")

Email sent successfully.


In [8]:
pip install XlsxWriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
conn.close()